In [ ]:
# importing the libraries
import torch, torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def show(tensor, num=25, wandbactive=0, name=''):
  data = tensor.detach().cpu()
  grid = make_grid(data[:num], nrow=5).permute(1,2,0)

  ## optional
  if (wandbactive==1):
    wandb.log({name:wandb.Image(grid.numpy().clip(0,1))})

  plt.imshow(grid.clip(0,1))
  plt.show()

### hyperparameters and general parameters
n_epochs=10000
batch_size=128
lr=1e-4
z_dim=200
device='cuda' #GPU

cur_step=0
crit_cycles=5
gen_losses=[]
crit_losses=[]
show_step=35
save_step=35

wandbact=1

C:\Users\suvod\Anaconda\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
#### optional
!pip install wandb -qqq
import wandb

wandb.login(key='108ce43c699626ffceae7d5b1d1a779f9dfae981')

wandb: Currently logged in as: suvodeepchowdhury108 (personall-projects). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\suvod/.netrc


True

In [ ]:
%%capture
experiment_name = wandb.util.generate_id()

myrun=wandb.init(
    project="wgan",
    group=experiment_name,
    config={
        "optimizer":"adam",
        "model":"wgan gp",
        "epoch":"1000",
        "batch_size":128
    }
)

config=wandb.config


In [ ]:
# generator model

class Generator(nn.Module):
    def __init__(self,z_dim =64, h_dim = 16):
        super(Generator,self).__init__()
        self.z_dim = z_dim

        self.gen  = nn.Sequential(

            nn.ConvTranspose2d(z_dim ,h_dim * 32, 4,2,0),
            nn.BatchNorm2d(h_dim * 32),
            nn.ReLU(True),

            nn.ConvTranspose2d(h_dim * 32, h_dim * 16, 4,2,1),
            nn.BatchNorm2d(h_dim * 16),
            nn.ReLU(True),

            nn.ConvTranspose2d(h_dim * 16, h_dim * 8, 4,2,1),
            nn.BatchNorm2d(h_dim * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(h_dim * 8, h_dim * 4, 4,2,1),
            nn.BatchNorm2d(h_dim * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(h_dim * 4, h_dim * 2,4,2,1),
            nn.BatchNorm2d(h_dim * 2),
            nn.ReLU(),

            nn.ConvTranspose2d(h_dim * 2, 3,4,2,1),
            nn.Tanh(),
        )


    def forward(self,nosie):
        x = noise.view(len(noise),self.z_dim, 1,1)
        return self.gen(x)

In [ ]:
def genNoise(num,z_dim ):
    return torch.randn(num,z_dim)

In [ ]:
## critic model
class Critic(nn.Module):
    def __init__(self,h_dim  = 16):
        super(Critic,self).__init__()
        self.crit = nn.Sequential(
            nn.Conv2d(3, h_dim, 4,2,1),
            nn.BatchNorm2d(h_dim),
            nn.LeakyReLU(0.2),


            nn.Conv2d(h_dim, h_dim * 2, 4,2,1),
            nn.BatchNorm2d(h_dim * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(h_dim * 2, h_dim * 4, 4,2,1),
            nn.BatchNorm2d(h_dim * 4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(h_dim * 4, h_dim * 8 , 4,2,1),
            nn.BatchNorm2d(h_dim * 8),
            nn.LeakyReLU(0.2),

            nn.Conv2d(h_dim * 8, h_dim * 16,4,2,1),
            nn.BatchNorm2d(h_dim * 16),
            nn.LeakyReLU(0.2),

            nn.Conv2d(h_dim * 16, 1,4,2,0)
        )

    def forward(self,image):
        crit_pred = self.crit(image)
        return crit_pred.view(len(crit_pred),-1)

In [ ]:
## optional, init your weights in different ways
def init_weights(m):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        torch.nn.init.normal_(m.weights, 0.0, 0.02)
        torch.nn.init.constant_(m.bais, 0)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weights, 0.0, 0.02)
        torch.nn.init.constant_(m.bais, 0)


In [ ]:
### Dataset, DataLoader, declare gen,crit, test dataset
class Dataset(Dataset):
    def __init__(self,path,size = 128, lim = 10000):
        self.sizes = [size,size]
        items,labels = [] , []

        for data in os.listdir(path)[:lim]:
            item = os.path.join(path,data)
            items.append(item)
            labels.append(data)
        self.items = items
        self.labels = labels

    def __len__(self):
        return len(self.items)

    def __getitem__(self,idx):
        data = PIL.Image.open(self.items[idx]).convert('RGB')
        data = np.asarray(torchvision.transforms.Resize(self.sizes)(data))
        data = np.transpose(data, (2,0,1)).astype(np.float32, copy = False)
        data = torch.from_numpy(data).div(255)
        return data, self.labels[idx]

In [ ]:
## Dataset path
extracted_folder_path = r"C:\Users\suvod\Downloads\img_align_celeba (4)\img_align_celeba"
ds = Dataset(extracted_folder_path, size = 128, lim = 10000)


In [ ]:
## DataLoader
dataloader = DataLoader(ds,batch_size = batch_size,shuffle = True)

In [ ]:
## Models
gen = Generator(z_dim)
crit = Critic()

In [ ]:
## Optimizers
gen_opt = torch.optim.Adam(gen.parameters() , lr=lr, betas = (0.5,0.9))
crit_opt = torch.optim.Adam(crit.parameters() , lr = lr , betas = (0.5, 0.9))

In [ ]:
## gradient penalty calculation

def get_gp(real,fake,crit,alpha,gamma = 10):
    mix_images = real * alpha + fake * (1-alpha)
    mix_scores = crit(mix_images)

    gradient = torch.autograd.grad(
        inputs = mix_images,
        outputs = mix_scores,
        grad_outputs = torch.ones_like(mix_scores),
        retain_graph = True,
        create_graph = True,
    )[0]

    gradient = gradient.view(len(gradient),-1)
    gp_norm = gradient.norm(2, dim = 1)
    gp = gamma *((gp_norm - 1) ** 2).mean()
    return gp

In [ ]:
## Save checkpoints
root_path = './data/'
def save_checkpoint(name):

    torch.save({
        'epoch':epoch,
        'model_state_dict':gen.state_dict(),
        'optimizer_state_dict': gen_opt.state_dict(),
    },f'{root_path}G-{name}.pkl')

    torch.save({
        'epoch':epoch,
        'model_state_dict':crit.state_dict(),
        'optimizer_state_dict': crit_opt.state_dict(),
    },f'{root_path}C-{name}.pkl')

    print("Saving Checkpoint")


In [ ]:
#Load checkpoints
def load_checkpoint(name):
    checkpoint = torch.load(f'{root_path}G-{name}.pkl')
    gen.load_state_dict(checkpoint['model_state_dict'])
    gen_opt.load_state_dict(checkpoint['optimizer_state_dict'])

    chekcpoint = torch.load(f'{root_path}C-{name}.pkl')
    crit.load_state_dict(checkpoint['model_state_dict'])
    crit_opt.load_state_dict(checkpointnt['optimizer_state_dict'])

    print('Loading Checkpoint')

In [ ]:
## Training loop

for epoch in range(n_epochs):
    for real,_ in tqdm(dataloader):
        cur_bs = len(real)

        ### CRITIC
        mean_crit_loss = 0
        for _ in range(crit_cycles):
            crit_opt.zero_grad()

            noise = genNoise(cur_bs,z_dim)
            fake = gen(noise)
            crit_pred_fake = crit(fake.detach())
            crit_pred_real = crit(real)

            alpha = torch.rand(len(real),1,1,1,device = "cpu",requires_grad = True)
            gp = get_gp(real,fake.detach(),crit,alpha)

            crit_loss = crit_pred_fake.mean() - crit_pred_real.mean() + gp
            mean_crit_loss += crit_loss.item() / crit_cycles


            crit_loss.backward(retain_graph = True)
            crit_opt.step()
        crit_losses += [mean_crit_loss]

        ### GENERATOR
        gen_opt.zero_grad()
        noise = genNoise(cur_bs, z_dim)
        fake = gen(noise)
        crit_fake_pred = crit(fake)

        gen_loss = -crit_fake_pred.mean()
        gen_loss.backward()
        gen_opt.step()

        gen_losses += [gen_loss.item()]


        ### Stats

        if (wandbact==1):
          wandb.log({'Epoch': epoch, 'Step': cur_step, 'Critic loss':mean_crit_loss, 'Gen loss': gen_loss})

        if cur_step % save_step == 0 and cur_step > 0:
          print("Saving checkpoint: ", cur_step, save_step)
          save_checkpoint("latest")

        if (cur_step % show_step == 0 and cur_step > 0):
          show(fake, wandbactive=1, name='fake')
          show(real, wandbactive=1, name='real')

          gen_mean=sum(gen_losses[-show_step:]) / show_step
          crit_mean = sum(crit_losses[-show_step:]) / show_step
          print(f"Epoch: {epoch}: Step {cur_step}: Generator loss: {gen_mean}, critic loss: {crit_mean}")

          plt.plot(
              range(len(gen_losses)),
              torch.Tensor(gen_losses),
              label="Generator Loss"
          )

          plt.plot(
              range(len(gen_losses)),
              torch.Tensor(crit_losses),
              label="Critic Loss"
          )

          plt.ylim(-150,150)
          plt.legend()
          plt.show()

    cur_step+=1


  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]